In [ ]:
# ==================================================
# 安装 unsloth 与指定版本 vllm 的依赖。
# ==================================================

# pip install unsloth vllm==0.8.5.post1

In [ ]:
# ==================================================
# 安装 langid 库，用于语言检测。
# ==================================================

# pip install langid -qq

In [1]:
# ==================================================
# 导入 Unsloath 的 FastLanguageModel，并加载本地 Qwen3-8B 模型；同时设置最大序列长度与 LoRA Rank。
# ==================================================

from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # 对于reasoning场景下，可以设置的更长
lora_rank = 32 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-03 15:42:45 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-03 15:42:45 [__init__.py:239] Automatically detected platform cuda.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


==((====))==  Unsloth 2025.6.12: Fast Qwen3 patching. Transformers: 4.52.4. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.151 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}
Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading /opt/tiger/test0/DeepSeek-R1-0528-Qwen3-8B with actual GPU utilization = 69.6%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 79.15 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 39.67 GB. Also swap space = 6 GB.
INFO 07-03 15:44:57 [config.py:717] This model supports multiple tasks: {'reward', 'score', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 07-03 15:44:57 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=1024.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'fp4', 'bnb_4bit_use_double_quant': False, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': [], 'llm_int8_threshold': 6.0}
INFO 07-03 15:44:58 [core.py:58] Initializing

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-03 15:45:02 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 07-03 15:45:03 [gpu_model_runner.py:1347] Model loading took 5.9071 GiB and 4.323834 seconds
INFO 07-03 15:45:21 [backends.py:420] Using cache directory: /home/tiger/.cache/vllm/torch_compile_cache/aa9982d8bb/rank_0_0 for vLLM's torch.compile
INFO 07-03 15:45:21 [backends.py:430] Dynamo bytecode transform time: 18.10 s
INFO 07-03 15:45:33 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 10.438 s
INFO 07-03 15:45:38 [monitor.py:33] torch.compile takes 18.10 s in total
INFO 07-03 15:45:38 [kv_cache_utils.py:634] GPU KV cache size: 341,328 tokens
INFO 07-03 15:45:38 [kv_cache_utils.py:637] Maximum concurrency for 1,024 tokens per request: 333.33x
INFO 07-03 15:46:32 [gpu_model_runner.py:1686] Graph capturing finished in 54 secs, took 1.64 GiB
INFO 07-03 15:46:32 [core.py:159] init engine (profile, create kv cache, warmup model) took 89.48 seconds
Unsloth: Just some info: will 

Unsloth 2025.6.12 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [ ]:
# ==================================================
# 通过 huggingface-cli 从 Hugging Face 下载数据集文件至本地路径。
# ==================================================

huggingface-cli download --resume-download --repo-type dataset open-r1/DAPO-Math-17k-Processed --local-dir open-r1/DAPO-Math-17k-Processed

### GRPO 聊天模板

In [2]:
# ==================================================
# 初始化聊天模板的相关标记变量，如 reasoning_start、reasoning_end、user_token 等。
# ==================================================

reasoning_start = None
reasoning_end = None
user_token = None
assistant_token = None

for token in tokenizer.get_added_vocab().keys():
    if "think" in token and "/" in token:
        reasoning_end = token
    elif "think" in token:
        reasoning_start = token
    elif "user" in token:
        user_token = token
    elif "assistant" in token:
        assistant_token = token

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
You must think in Bahasa Indonesia."""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nYou must think in Bahasa Indonesia.'

In [3]:
# ==================================================
# 演示使用 tokenizer.apply_chat_template 将多轮消息转换为模型可用的 token。
# ==================================================

print(tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"<think>I think it's 2.2</think>2"},
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"<think>I think it's 2.2</think>2"},
], tokenize = False, add_generation_prompt = True))

<｜begin▁of▁sentence｜><｜User｜>What is 1+1?<｜Assistant｜>2<｜end▁of▁sentence｜><｜User｜>What is 1+1?<｜Assistant｜>2<｜end▁of▁sentence｜><｜Assistant｜>


### 数据准备
<a name="Data"></a>

我们将使用 Hugging Face 的 [Open R1 Math 数据集](https://huggingface.co/datasets/openr1) 以及 [GSM8K 数据集](https://huggingface.co/datasets/openai/gsm8k)。

In [4]:
# ==================================================
# 加载 Open R1 Math 数据集，并查看数据集对象。
# ==================================================

from datasets import load_dataset
dataset = load_dataset("/opt/tiger/test0/open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

Generating train split:   0%|          | 0/14116 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'solution', 'data_source', 'source_prompt', 'ability', 'reward_model', 'extra_info'],
    num_rows: 14116
})

先查看第一行数据：

In [5]:
# ==================================================
# 打印首条样本的用户 prompt。
# ==================================================

dataset[0]["prompt"]

'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.'

In [6]:
# ==================================================
# 打印首条样本的标准答案 solution。
# ==================================================

dataset[0]["solution"]

'34'

在 GSM8K 中，我们注意到所有答案都包含 `####`，因此需要额外提取。但对于 Open R1 数据集，可以跳过这一步。

In [7]:
# ==================================================
# 定义 extract_hash_answer 函数，用于从答案字符串中提取 '####' 后的数字；随后进行示例验证。
# ==================================================

def extract_hash_answer(text):
    # if "####" not in text: return None
    # return text.split("####")[1].strip()
    return text
extract_hash_answer(dataset[0]["solution"])

'34'

让我们映射数据集并查看第一行：

In [8]:
# ==================================================
# 对数据集进行 map 转换，生成带系统提示、用户提示及提取后的答案字段的新数据集。
# ==================================================

dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["prompt"]},
    ],
    "answer": extract_hash_answer(x["solution"]),
})
dataset[0]

Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

{'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nYou must think in Bahasa Indonesia.',
   'role': 'system'},
  {'content': 'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.',
   'role': 'user'}],
 'solution': '34',
 'data_source': 'math_dapo',
 'source_prompt': [{'content': 'Solve the following math problem step by step. The last line of your response should be of the form Answer: $Answer (without quotes) where $Answer is the answer to the problem.\n\nIn triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angl

我们创建正则表达式，用来匹配推理部分与答案：

In [9]:
# ==================================================
# 引入正则表达式，定义 solution_end_regex 以及辅助语言检测函数。
# ==================================================

import re

# Add optional EOS token matching
solution_end_regex = rf"{reasoning_end}(.*)"

match_format = re.compile(solution_end_regex, re.DOTALL)
match_format

re.compile(r'</think>(.*)', re.DOTALL|re.UNICODE)

验证正则表达式是否有效：

In [10]:
# ==================================================
# 使用 match_format 正则，对示例字符串进行格式匹配测试（未加 <think> 标签）。
# ==================================================

match_format.findall(
    "Let me think!</think>"\
    f"Hence, the solution is 2.",
)

['Hence, the solution is 2.']

In [11]:
# ==================================================
# 使用 match_format 正则，对示例字符串进行格式匹配测试（含 <think> 标签）。
# ==================================================

match_format.findall(
    "<think>Let me think!</think>"\
    f"\n\nHence, the solution is 2",
)

['\n\nHence, the solution is 2']

接下来我们要创建一个奖励函数：若输出格式完全正确则给 3 分：

In [12]:
# ==================================================
# 定义 match_format_exactly 函数：若返回内容严格符合模板格式，则给予高奖励。
# ==================================================

def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

若完全匹配失败，则按符号数量部分奖励模型：

In [13]:
# ==================================================
# 定义 match_format_approximately 函数：当模板部分符合时，以符号数量作为次级奖励。
# ==================================================

def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        scores.append(score)
    return scores

提取模型生成的答案，根据其与真实答案的接近程度按比值奖励或惩罚：

In [14]:
# ==================================================
# 定义 check_answer 函数：从预测结果里抽取答案，与真值比较并计算分数。
# ==================================================

def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue
        # Correct answer gets 5 points!
        if guess == true_answer:
            score += 5.0
        # Match if spaces are seen, but less reward
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2: score += 1.5
                else: score -= 2.5 # Penalize wrong answers
            except:
                score -= 4.5 # Penalize
        scores.append(score)
    return scores

有时答案不止一个数字，例如 20。

我们还会移除类似 123,456 中的逗号分隔符。

In [15]:
# ==================================================
# 测试 match_numbers 正则表达式在不同数字字符串上的提取效果。
# ==================================================

match_numbers = re.compile(
    r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("  0.34  "))
print(match_numbers.findall("  123,456  "))
print(match_numbers.findall("  -0.234  "))
print(match_numbers.findall("17"))

['0.34']
['123,456']
['-0.234']
['17']


最后，我们引入 DeepSeek R1 论文中的 `language consistency reward`，以确保思考过程保持一致的目标语言。

In [16]:
# ==================================================
# 使用 langid 库定义 get_lang 函数，检测输入文本的语言类别。
# ==================================================

import langid

def get_lang(text: str) -> str:
    if not text:
        return "und"
    lang, _ = langid.classify(text)
    return lang


print(get_lang("Hello, How are you")) # This should return en
print(get_lang("Aku berpikir kalau aku adalah kamu")) # This should return id
print(get_lang("我在这里")) # This should return zh

[2025-07-03 15:54:32] INFO langid.py:162: initializing identifier


en
id
zh


In [17]:
# ==================================================
# 定义 format_and_language_reward_func：综合格式匹配与目标语言一致性进行奖励。
# ==================================================

import re

def format_and_language_reward_func(completions, **kwargs):
    scores = []

    for completion_item in completions:
        if not completion_item or not isinstance(completion_item[0], dict) or "content" not in completion_item[0]:
            scores.append(-5.0)
            print(f"Warning: Malformed completion item, assigning default low score: {completion_item}")
            continue

        content = completion_item[0]["content"]

        lang = get_lang(content)

        if lang == 'id':
            score = 5.0
        elif lang == 'en':
            score = -3.0
        elif lang == 'zh':
            score = -3.0
        else:
            score = -5.0

        scores.append(score)

    return scores

In [18]:
# ==================================================
# 示例：调用 format_and_language_reward_func 并打印得分结果。
# ==================================================

prompts = [
    [{"role": "assistant", "content": "What is the result of (1 + 2) * 4?"}],
    [{"role": "assistant", "content": "What is the result of (3 + 1) * 2?"}],
]
completions = [
    [{"role": "assistant", "content": "<think>The sum of 1 and 2 is 3, which we multiply by 4 to get 12.</think><answer>(1 + 2) * 4 = 12</answer>"}],
    [{"role": "assistant", "content": "The sum of 3 and 1 is 4, which we multiply by 2 to get 8. So (3 + 1) * 2 = 8."}],
]
format_and_language_reward_func(prompts=prompts, completions=completions)

[-3.0, -3.0]

准备主函数：打印生成答案，并将文本转换为浮点数以比较是否正确。

In [19]:
# ==================================================
# 定义 check_numbers 函数：比较预测答案与真实数字之间的误差，并打印日志。
# ==================================================

global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

获取提示长度的 90% 分位数，避免过长的提示被截断（删除最前 10% 的超长提示）。

In [20]:
# ==================================================
# 将数据集文本映射为 tokens，准备训练用的输入。
# ==================================================

tokenized = dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

<｜begin▁of▁sentence｜>You are given a problem.
Think about the problem and provide your working out.
You must think in Bahasa Indonesia.<｜User｜>In triangle $ABC$, $\sin \angle A = \frac{4}{5}$ and $\angle A < 90^\circ$. Let $D$ be a point outside triangle $ABC$ such that $\angle BAD = \angle DAC$ and $\angle BDC = 90^\circ$. Suppose that $AD = 1$ and that $\frac{BD}{CD} = \frac{3}{2}$. If $AB + AC$ can be expressed in the form $\frac{a\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.<｜Assistant｜>


Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

Max Length =  180


<a name="Train"></a>
### 训练模型

现在配置 GRPO Trainer 及所有参数！

In [21]:
# ==================================================
# 设置 vLLM 的采样参数，例如 temperature、top_k、最大生成长度等。
# ==================================================

max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "swanlab", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


开始运行 Trainer！向上滚动可查看训练过程中生成的表格日志。

In [22]:
# ==================================================
# 配置并实例化 GRPOTrainer，其中包含若干奖励函数与训练参数。
# ==================================================

# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
        format_and_language_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

[2025-07-03 15:56:05] WARNING other.py:492: Detected kernel version 5.4.143, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,728 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 87,293,952 of 8,000,000,000 (1.09% trained)


swanlab: Tracking run with swanlab version 0.6.4                                   
swanlab: Run data will be saved locally in /opt/tiger/test0/swanlog/run-20250703_155610-0e8cd89d
swanlab: 👋 Hi twosugar, welcome to swanlab!
swanlab: Syncing run outputs to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@twosugar/test0
swanlab: 🚀 View run at https://swanlab.cn/@twosugar/test0/runs/veyt1tdlqckdz81hdyqmc


********************Question:
In the diagram, each of the three identical circles touch the other two.  The circumference of each circle is 36.  What is the perimeter of the shaded region? [asy]

defaultpen(1);

path p = (1, 0){down}..{-dir(30)}dir(-60){dir(30)}..{dir(-30)}((2, 0) + dir(-120)){-dir(-30)}..{up}(1, 0)--cycle;
fill(p, gray(0.75));

draw(unitcircle);
draw(shift(2 * dir(-60)) * unitcircle);
draw(shift(2) * unitcircle);
[/asy] 
Answer:
18 
Response:
<think>
Pertama, soalnya: Ada tiga lingkaran identik yang saling menyentuh. Keliling setiap lingkaran adalah 36. We're asked for the perimeter of the shaded region, dan dari Asymptote diagram, tampaknya shaded region adalah daerah yang diisi oleh kurva ketiga lingkaran.

Jadi, ada tiga lingkaran yang sama besar, dan masing-masing menyentuh lingkaran lainnya. Ini menimbulkan pertanyaan: di mana persentuhan ini terjadi? Dalam diagram Asymptote, tampaknya lingkaran berada di posisi yang membentuk suatu pola segitiga. Langsung saja, 

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std,rewards / format_and_language_reward_func / mean,rewards / format_and_language_reward_func / std
1,0.000000,-1.500000,4.618802,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,0.000000,0.000000,1.000000,4.618802
2,0.000000,0.500000,4.000000,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,0.000000,0.000000,3.000000,4.000000
3,0.000000,2.500000,0.000000,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000318,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,0.000000,0.000000,5.000000,0.000000
4,0.000000,-5.875000,0.750000,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000994,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,-0.375000,0.750000,-3.000000,0.000000
5,0.000000,1.750000,0.866025,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000478,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,-0.750000,0.866025,5.000000,0.000000
6,0.000000,-7.000000,0.000000,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.001132,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,-1.500000,0.000000,-3.000000,0.000000
7,0.000000,-7.000000,0.000000,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000867,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,-1.500000,0.000000,-3.000000,0.000000
8,0.000000,1.000000,0.000000,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000578,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,-1.500000,0.000000,5.000000,0.000000
9,0.000000,-7.000000,0.000000,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000550,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,-1.500000,0.000000,-3.000000,0.000000
10,0.000000,-0.250000,4.555217,843.000000,843.000000,843.000000,1.000000,0.000000,0.000000,0.000000,0.000736,0.000000,0.000000,-0.500000,0.000000,-2.000000,0.000000,-0.750000,0.866025,3.000000,4.000000


Unsloth: Will smartly offload gradients to save VRAM!
********************Question:
In a class of 20 students, all but 4 of the students put their names on a typed assignment. If the teacher randomly guesses, what is the probability that she correctly guesses which paper belongs to each of the four remaining students? Express your answer as a common fraction.The answer is in the form 
rac{m}{n}, where gcd(m, n) = 1. Please provide the value of m + n. 
Answer:
25 
Response:
<think>
The problem states there are 20 students, and all but 4 put their names on their assignments. So, 16 students have their names on their papers, and 4 do not. The teacher guesses randomly, and I need to find the probability that she correctly identifies which paper belongs to each of the four students who didn't put their names on.

First, I need to understand what's being asked. The teacher is guessing which paper is whose for these four students, but since the papers are unlabeled, I assume she's assigning 

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


TrainOutput(global_step=100, training_loss=7.181387914911853e-07, metrics={'train_runtime': 4359.8674, 'train_samples_per_second': 0.092, 'train_steps_per_second': 0.023, 'total_flos': 0.0, 'train_loss': 7.181387914911853e-07})

<a name="Inference"></a>
### 推理

现在测试训练后的模型！首先测试未使用 GRPO 的基线模型：

In [23]:
# ==================================================
# 使用 fast_generate 测试模型在单条输入上的推理输出。
# ==================================================

text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

" - The Free Online Math Book - BookSummitFree online math books, free online science books, free online programming books, and nonfiction books free online.\nSkip to content\nSearch for:\nSearch\nMenu\nMenu\nHome\nBooks\nPrograming\nMath\nScience\nAbout\nContact\nDonations\nAuthor: BookSummit\nThe Free Online Math Book\nAn independent publishing platform for math books and e-books online.\nFree Online Math Book > Calculus (Single Variable), Free Online Books\nWhat is the sqrt of 101?\nadmin June 23, 2022 Leave a comment\nsqrt(101) = ? Here we can see the solution step by step.\nFirst, find out what 10^2 and 11^2 gives.\n10^2 = 100\n11^2 = 121\nSo, 10 and 11 are the two numbers. And 100 and 121 are the squares.\nBut 100 is closer to 101, so sqrt(101) is approximately 10.something.\n\nSo, let me do the calculation for the square root of 101.\n\nLet me take a=10, b=101. Then, sqrt(101) = sqrt(b) = a + (b - a^2)/(2a) for initial approximation.\n\nBut a^2 is 100, and b is 101, so (b - a^2)

现在使用我们刚用 GRPO 训练得到的 LoRA，首先保存 LoRA 权重：

In [24]:
# ==================================================
# 保存 LoRA 适配器权重到 'grpo_lora' 目录。
# ==================================================

model.save_lora("grpo_lora")

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


验证 LoRA 是否确实被训练：

In [25]:
# ==================================================
# 使用 safetensors 检查 LoRA 权重文件中的张量是否成功训练（非全零）。
# ==================================================

from safetensors import safe_open

tensors = {}
with safe_open("grpo_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

加载 LoRA 并进行测试，此处不使用自定义 system prompt，以尽量保持模型原有的推理能力：

In [26]:
# ==================================================
# 示例：不带 system prompt，使用 LoRA 进行推理。
# ==================================================

messages = [
    {"role": "user",   "content": "Solve (x + 2)^2 = 0"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_lora"),
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'<think>\nI have this equation to solve: (x + 2)^2 = 0. I need to find the value of x that makes this true.\n\nThis equation says that (x + 2) squared equals zero. Since it\'s a square equal to zero, I know that only zero raised to any power is zero. So, for (something)^2 to be zero, that "something" must be zero.\n\nIn this case, the something is (x + 2), so x + 2 must be zero.\n\nTherefore, x + 2 = 0.\n\nNow, solving for x, subtract 2 from both sides: x = -2.\n\nI can think about this as the equation (x + 2)^2 = 0, which means x + 2 = 0 because if a square is zero, the thing being squared must be zero.\n\nIf I expand this, (x + 2)^2 = x^2 + 4x + 4, and set that equal to zero: x^2 + 4x + 4 = 0.\n\nNow, factor this quadratic. I know it factors to (x + 2)(x + 2) = 0, or (x + 2)^2 = 0.\n\nSo again, x + 2 = 0, so x = -2.\n\nUsing the quadratic formula: x = [-b ± sqrt(b^2 - 4ac)] / 2a, where a = 1, b = 4, c = 4.\n\nDiscriminants: b^2 - 4ac = 16 - 16 = 0, so sqrt(0) = 0, so x = [-4 ± 0]/2 =

接下来使用带有新语言 system prompt 的方式进行测试：

In [27]:
# ==================================================
# 示例：带 system prompt，但不使用 LoRA 进行推理。
# ==================================================

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "Solve (x + 2)^2 = 0"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_lora"),
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'<think>\nOkay, user wants me to solve (x + 2)^2 = 0. But they also specified that I must process this in Indonesian. Interesting constraint.\n\nFirst, let\'s analyze the equation. This is a simple quadratic where x+2 is squared and equals zero. I remember that squaring any number gives zero only if that number is zero itself. So (x+2) must be zero.\n\nBut wait, the user said this must be done in Indonesian. They want my thought process in Bahasa Indonesia. Hmm, I need to switch all internal reasoning to Indonesian temporarily to satisfy the constraint. But the math logic will be the same.\n\nBreaking it down: the equation is a perfect square set to zero. The solution is straightforward. I should explain the reasoning step by step since the user asked for "working out". \n\nFirst, we can take square root both sides. Remember, square root gives both positive and negative options? Wait no, since square root is defined to be non-negative, but in equations we consider both positive and neg

对比带 system prompt 但不加 LoRA 的结果：

In [28]:
# ==================================================
# 示例：带 system prompt 并加载 LoRA 后进行推理。
# ==================================================

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "Solve (x + 2)^2 = 0"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'<think>\nSaya perlu menyelesaikan persamaan (x + 2)^2 = 0. Persamaan ini terlihat sederhana. Karena ini adalah persamaan kuadrat, tetapi disajikan dalam bentuk kuadrat sempurna.\n\nSaya ingat bahwa (a + b)^2 = a^2 + 2ab + b^2, tapi di sini, (x + 2)^2 = 0. Jadi, mari saya selesaikan dengan mencari nilai x yang memenuhi persamaan tersebut.\n\nJika (x + 2)^2 = 0, maka x + 2 harus sama dengan 0, karena hanya bilangannya sendiri yang mengalikan dirinya sendiri hasilnya nol, yaitu angka nol itu sendiri.\n\nJadi, x + 2 = 0. Kemudian, x = -2.\n\nSaya bisa memeriksa jawaban ini. Substitusi x = -2 ke dalam persamaan.\n\n(x + 2)^2 = (-2 + 2)^2 = (0)^2 = 0, ya, memang benar.\n\nApakah ada solusi lain? Dalam persamaan kuadrat biasanya ada dua solusi, tapi di sini hanya satu solusi karena ini adalah bentuk (something)^2 = 0, yang hanya memiliki satu akar.\n\nSaya tahu itu sifat akar kuadrat, bahwa jika sesuatu kuadrat sama dengan nol, maka sesuatu itu harus nol.\n\nJadi, tunggal solusinya.\n\nPerta

从数据集中抽取 20 个样本，比较使用 LoRA 与不使用 LoRA 的正确语言输出数量：

In [29]:
# ==================================================
# 随机抽取 20 条样本组成 sample_dataset 供后续评估。
# ==================================================

sample_dataset = dataset.shuffle(seed = 3407).select(range(20))
sample_dataset

Dataset({
    features: ['prompt', 'solution', 'data_source', 'source_prompt', 'ability', 'reward_model', 'extra_info', 'answer'],
    num_rows: 20
})

In [30]:
# ==================================================
# 在 20 条样本上统计使用与未使用 LoRA 时目标语言输出数量的差异。
# ==================================================

with_lora_id_count = 0
without_lora_id_count = 0

print("Comparing language usage with and without LoRA on 20 samples:")
print("=" * 60)

for i, sample in enumerate(sample_dataset):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": sample["prompt"][1]["content"]},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )

    output_with_lora = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=model.load_lora("grpo_lora"),
    )[0].outputs[0].text

    output_without_lora = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=None,
    )[0].outputs[0].text

    lang_with_lora = get_lang(output_with_lora)
    lang_without_lora = get_lang(output_without_lora)

    if lang_with_lora == 'id':
        with_lora_id_count += 1
    if lang_without_lora == 'id':
        without_lora_id_count += 1

    # Print progress every 5 samples
    if (i + 1) % 5 == 0:
        print(f"Processed {i + 1}/20 samples...")

print("\n" + "=" * 60)
print("RESULTS:")
print(f"With LoRA - Indonesian responses: {with_lora_id_count}/20 ({with_lora_id_count/20*100:.1f}%)")
print(f"Without LoRA - Indonesian responses: {without_lora_id_count}/20 ({without_lora_id_count/20*100:.1f}%)")
print(f"Improvement: +{with_lora_id_count - without_lora_id_count} Indonesian responses with LoRA")

Comparing language usage with and without LoRA on 20 samples:


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed 20/20 samples...

RESULTS:
With LoRA - Indonesian responses: 12/20 (60.0%)
Without LoRA - Indonesian responses: 8/20 (40.0%)
Improvement: +4 Indonesian responses with LoRA


我们的推理模型显著更好——虽然并非总是正确，只训练了约一小时；如果扩展序列长度并训练更久，效果会更好！

In [ ]:
# ==================================================
# （可选）将模型合并保存为 16 位权重。
# ==================================================

# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


In [ ]:
# ==================================================
# （可选）将模型保存为 8-bit GGUF 量化格式，例如 Q8_0，并提示用户获取 HF token。
# ==================================================

# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )